In [ ]:
#!/usr/bin/env python

"""
Example classifier on Numerai data using a logistic regression classifier.
To get started, install the required packages: pip install pandas, numpy, sklearn
"""

import pandas as pd
import numpy as np
from sklearn import metrics, preprocessing, linear_model


def main():
    # Set seed for reproducibility
    np.random.seed(0)

    print("Loading data...")
    # Load the data from the CSV files
    ''' training data contains only training data'''
    training_data = pd.read_csv('in/numerai_training_data.csv', header=0)
    '''
    prediction_data contains both validation data (with targets) and test data
    (without targets)
    '''
    prediction_data = pd.read_csv('in/numerai_tournament_data.csv', header=0)

    # print('\n')
    # print('prediction_data')
    # print(prediction_data)


    # Transform the loaded CSV data into numpy arrays
    '''
    Goes through all the pd.DataFrame columns titles. If they contain 'feature'
    they are added to the features list
    '''
    features = [f for f in list(training_data) if "feature" in f]
    X = training_data[features]                     # pd.DataFrame of all training features
    Y = training_data["target"]                     # pd.Series of the classes
    x_prediction = prediction_data[features]        # pd.DataFrame of all validation and test features
    ids = prediction_data["id"]
    print('\n')
    print('ids')
    print(ids)
    print('\n')
    print('ids')
    print(ids.duplicated)

    # This is your model that will learn to predict
    model = linear_model.LogisticRegression(n_jobs=-1)

    print("Training...")
    # Your model is trained on the training_data
    model.fit(X, Y)

    print("Predicting...")
    # Your trained model is now used to make predictions on the numerai_tournament_data
    # The model returns two columns: [probability of 0, probability of 1]
    # We are just interested in the probability that the target is 1.
    y_prediction = model.predict_proba(x_prediction)
    results = y_prediction[:, 1]
    results_df = pd.DataFrame(data={'probability':results})
    joined = pd.DataFrame(ids).join(results_df)

    print("Writing predictions to predictions.csv")
    # Save the predictions out to a CSV file
    joined.to_csv("predictions.csv", index=False)
    # Now you can upload these predictions on numer.ai


if __name__ == '__main__':
    main()


In [ ]:
project = 'test_datasets'

import os.path
import sys
current_dir = os.path.abspath('./')
project_dir = current_dir[:current_dir.rfind(project)+len(project)+1]
sys.path.insert(0, project_dir)

In [ ]:
training_data = pd.read_csv('in/numerai_training_data.csv', header=0)
prediction_data = pd.read_csv('in/numerai_tournament_data.csv', header=0)

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload
import numpy as np
import pandas as pd
from sklearn import preprocessing, feature_extraction, feature_selection, model_selection, metrics
import xgboost as xgb
import matplotlib.pyplot as plt
# import utils
# from classes import ML


class ML(object):
    def __init__(self, df=None, x_columns=None, y_column=None, convert_dict=None,
                 drop_dict=None, replace_dict=None, feature_dict=None,
                 method=None, model_params={}, *args, **kwargs):
        super(ML, self).__init__()
        self.df = df
        self.x_columns = x_columns
        self.y_column = y_column
        self.convert_dict = convert_dict
        self.drop_dict = drop_dict
        self.replace_dict = replace_dict
        self.feature_dict = feature_dict
        self.method = method
        self.model_params = model_params
        self.sample_weight=None
        self.eval_metric=None
        
        self.n_columns = len(self.df.columns)-2

    def convert(self):
        if self.convert_dict is not None:
            for column, func in self.convert_dict.items():
                self.df[column] = func(self.df[column])

    def feature(self):
        '''call feature building functions on columns'''
        for column, func in self.feature_dict.items():
            self.df[column] = func(self.df)

    def save_mapping(self):
        pass

    def load_mapping(self):
        pass

    def get_X(self, sparse=True):
        self.vectorizer_ = feature_extraction.DictVectorizer(sparse=sparse)
        self.X = self.vectorizer_.fit_transform(self.df[self.x_columns].to_dict(orient='records'))
        self.feature_columns_ = self.vectorizer_.vocabulary_
        return self

    def get_y(self):
        self.y = np.array(self.df[self.y_column])
        return self.y

    def create_model(self, method=None, model_params=None):
        if method:
            self.method = method
        
        if model_params:
            self.model_params = model_params

        if self.method == 'classification':
            if model_params:
                self.model = xgb.XGBClassifier(**self.model_params)
            else:
                self.model = xgb.XGBClassifier()
            return True
        elif self.method == 'regression':
            print('Not implemented yet')

    def resample(self):
        self.sampler = SMOTETomek(n_jobs=-1)
        self.X, self.y = self.sampler.fit_sample(self.X, self.y)

    def cross_val(self, scoring='accuracy', cv=4, groups=None, seed=42):
        cross_vals = model_selection.cross_val_score(self.model, self.X, self.y,
                                                     scoring=scoring, cv=cv,
                                                     groups=groups, n_jobs=-1)
        self.cross_vals = cross_vals
        return cross_vals #numpy.mean(cross_vals), numpy.min(cross_vals), numpy.max(cross_vals)
    
    def train(self, sample_weight=None, eval_metric=None):
#         if isinstance(self.model, XGBClassifier):
        if sample_weight:
            self.sample_weight = sample_weight

        if eval_metric:
            self.eval_metric = eval_metric
        else:
            self.eval_metric = 'error'

        self.model.fit(self.X, self.y, sample_weight=self.sample_weight, eval_metric=self.eval_metric)
        


    def pickle(self):
        pass

    def unpickle(self):
        pass

    def score(self):
        pass

    def describe():
        from IPython.display import display

        if self.df:
            self.df.describe()
            
        if self.model:
            print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# production code

In [ ]:
num_features = len(training_data.columns)-4
x_columns = ['feature'+str(i) for i in range(1,num_features)] + ['era_int']
y_column = 'target'

drop_dict = {}
replace_dict = {}
convert_dict = {'created': pd.to_datetime,
                     'planned_installation_time': pd.to_numeric,
                     'status_type': pd.to_numeric}

def get_end_of_month(df):
    return df['created'].dt.is_month_end

def get_era(df):
    return df['era'].apply(lambda s: int(s[3:]))

feature_dict = {'era_int': get_era}

ml = ML(df=training_data, 
           x_columns=x_columns,
           y_column=y_column,
           drop_dict=drop_dict,
           replace_dict=replace_dict,
           convert_dict=convert_dict,
           feature_dict=feature_dict)
ml.feature()
ml.get_X()
ml.get_y()
# ml.y
ml.df.head()

# working env

In [ ]:
# create cross_val
ml.create_model(method='classification')


def cross_val(self, scoring='accuracy', cv=4, group_col=None):
    if group_col:
        groups = self.df[group_col]
    else:
        groups = None
    cross_vals = model_selection.cross_val_score(self.model, self.X, self.y,
                                                 scoring=scoring, cv=cv,
                                                 groups=groups, n_jobs=-1)
    self.cross_vals = cross_vals
    
group_col='era'
# cross_val(ml, scoring='recall')
model_selection.cross_val_score?
ml.cross_vals

In [ ]:
# create train
import copy
m_ = copy.copy(ml)

m_.create_model(method='classification')
m_.model.fit(m_.X, m_.y, sample_weight=None, eval_set=None, eval_metric=None)

In [ ]:
ml.df.count()

In [ ]:
model_selection.cross_val_score?

In [ ]:
# feature selection cross val

In [ ]:
ml.create_model(method='classification')
ml.cross_val()
# ml.train()

In [ ]:
model.predict()

In [ ]:
# ml.train()
y_pred = ml.model.predict()
y_prob = ml.model.predict_proba()

In [ ]:
ml.create_model(method='classification')
model = ml.model
df = ml.df.copy()
X = ml.X.copy()
y = ml.y.copy()
# cross val:
scoring = 'accuracy'
cv = 4
groups='era'

# cross_vals = model_selection.cross_val_score(model, X, y, scoring=scoring, groups=None, cv=cv, n_jobs=-1)
# cross_vals
# model_selection.cross_val_score?

def train(self, sample_weight=None, eval_metric=None):
#         if isinstance(self.model, XGBClassifier):
    if sample_weight:
        self.sample_weight = sample_weight

    if eval_metric:
        self.eval_metric = eval_metric
    else:
        self.eval_metric = 'error'

    self.model.fit(self.X, self.y, sample_weight=self.sample_weight, eval_metric=self.eval_metric)
    
train(ml)
# y_pred = model.predict()

In [ ]:
ml.model

In [ ]:
model.fit(X, y, sample_weight=None, eval_metric=None)


In [ ]:
model.feature_importances_

In [ ]:
import graphviz
xgb.plot_tree(model)

In [ ]:
# unique index
df['id'].nunique()/len(df)

In [ ]:
for i in range(model.n_columns):
    print('feature'+str(i+1))
    df.sort_values(by='id').plot(kind='line', x='id', y='feature'+str(i+1))
    df.plot(x=df.index, y='feature'+str(i+1))
    plt.show()

# graveyard

In [ ]:
df.describe()